<a href="https://colab.research.google.com/github/ekaratnida/Applied-machine-learning/blob/master/Stock_forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.ultimatepython.co/post/python-yahoo-finance

In [ ]:
!pip install pandas
!pip install numpy
!pip install requests
!pip install lxml
!pip install yfinance

In [4]:
import yfinance as yf

In [6]:
mbk = yf.Ticker('mbk.bk')
interval = '1d'
start = '2022-1-1'
end = '2023-11-23'
mbk_price_df = mbk.history(interval='1d',start=start,end=end)
mbk_price_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-01-04 00:00:00+07:00,12.841338,13.126701,12.746217,13.031580,1975600,0.0,0.0
2022-01-05 00:00:00+07:00,13.031580,13.126701,12.936459,13.031580,553500,0.0,0.0
2022-01-06 00:00:00+07:00,13.031580,13.031580,12.746217,12.841338,769200,0.0,0.0
2022-01-07 00:00:00+07:00,12.746217,12.841338,12.651096,12.746217,616100,0.0,0.0
2022-01-10 00:00:00+07:00,12.746217,12.746217,12.555975,12.651096,666700,0.0,0.0
...,...,...,...,...,...,...,...
2023-09-01 00:00:00+07:00,18.500000,19.400000,18.500000,18.799999,7158000,0.0,0.0
2023-09-04 00:00:00+07:00,19.100000,19.600000,19.100000,19.299999,4836500,0.0,0.0
2023-09-05 00:00:00+07:00,19.400000,19.700001,19.299999,19.200001,2746300,0.0,0.0


In [2]:
from thaistock import SET

#สร้างตัวแปร stock (object)
stock = SET()

#เช็คราคา ณ ปัจจุบัน (ใช้ขณะตลาดเปิดได้)
current = stock.current('TEAMG',show=True,header=True)

#โชว์หัวข้อตารางราคาย้อนหลังเพื่ออ้างอิง index
stock.show_header()

'''
เลือก index ของคอลัมน์ที่ต้องการ
วันที่ [0]
ราคาเปิด [1]
ราคาสูงสุด [2]
ราคาต่ำสุด [3]
ราคาเฉลี่ย [4]
ราคาปิด [5]
เปลี่ยนแปลง [6]
%เปลี่ยนแปลง [7]
ปริมาณ(พันหุ้น) [8]
มูลค่า(ล้านบาท) [9]
SET Index [10]
%เปลี่ยนแปลง [11]

ต้องการแสดงวันที่ ,ราคาปิด, เปลี่ยนแปลง, %เปลี่ยนแปลง
stock.historical('CODEหุ้น', select=[0,5,6,7])

'''

# เช็คราคาย้อนหลัง 30 วัน แสดงวันที่ ,ราคาปิด, เปลี่ยนแปลง, %เปลี่ยนแปลง
historical_price = stock.historical('TEAMG',days=30,show=True,select=[0,5,6,7],header=True)

# พล็อตราคาปิด 30 วันย้อนหลัง
stock.plot('TEAMG',days=30)

URLError: ignored

In [ ]:
!pip install pandas_datareader
from pandas_datareader import data as pdr
from tqdm import tqdm,trange
import sys
stock_list = ['KBANK','SCB','BBL','KTB']
stock_data = []
stock_name = []
for quote in tqdm(stock_list):
    try:
        stock_data.append(pdr.get_data_yahoo(f'{quote}.BK', start='2010-01-01', end='2019-04-15'))
        stock_name.append(quote)
    except:
        print("Error:", sys.exc_info()[0])
        print("Description:", sys.exc_info()[1])

In [ ]:
stock_data[0].head()

In [ ]:
from matplotlib import pyplot
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

# function สำหรับ preprocess ข้อมูล time series หลายตัวแปร
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True,feat_name=None):

    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()

    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [f'{feat_name[j]}(t-{i})' for j in range(n_vars)]

    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [f'{feat_name[j]}(t)' for j in range(n_vars)]
        else:
            names += [f'{feat_name[j]}(t+{i})' for j in range(n_vars)]

    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names

    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# เลือกข้อมูลหุ้นจาก list ของ DataFrame และ drop column 'Close' เนื่องจากเราจะใช้ column 'Adj. Close' เท่านั้น
stock_data[0] = stock_data[0].drop('Close',axis=1)
dataset = stock_data[0]
values = dataset.values
values = values.astype('float32')

# ทำ scaling ข้อมูลด้วยวิธี min max scaling เปลี่ยน scale ข้อมูลแต่ละ column ให้อยู่ระหว่าง [0,1] และเก็บค่า min max แต่ละ column ไว้สำหรับทำ rescale ข้อมูลภายหลัง
min_dict = dict()
max_dict = dict()
for col in dataset.columns:
  min_dict[col] = dataset[col].min()
  max_dict[col] = dataset[col].max()
  dataset[col] = (dataset[col] - dataset[col].min())/(dataset[col].max()-dataset[col].min())

# ใช้ function สำหรับ preprocess ข้อมูลที่เขียนไว้ และ drop column ที่ไม่ได้ใช้
reframed = series_to_supervised(dataset.values, 30, 3,feat_name=stock_data[0].columns)
reframed.drop(['High(t)','High(t+1)','High(t+2)','Low(t)','Low(t+1)','Low(t+2)','Open(t)','Open(t+1)','Open(t+2)','Volume(t)','Volume(t+1)',\
               'Volume(t+2)'],axis=1,inplace=True)

In [ ]:
reframed.head()

In [ ]:
# split into train and test sets
values = reframed.values
n_train_percent = 0.9
split = int(values.shape[0]*0.9)
train = values[:split, :]
test = values[split:, :]

In [ ]:
date_reframed = dataset.index[reframed.index]
date_train = date_reframed[:split]
date_test = date_reframed[split:]

In [ ]:
# split into input and outputs
train_X, train_y = train[:, :-3], train[:, -3:]
test_X, test_y = test[:, :-3], test[:, -3:]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

In [ ]:
train_X

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(3))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=100, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)

In [ ]:
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))

In [ ]:
inv_yhat = yhat*(max_dict['Adj Close']-min_dict['Adj Close'])+min_dict['Adj Close']
inv_y = test_y*(max_dict['Adj Close']-min_dict['Adj Close'])+min_dict['Adj Close']

In [ ]:
import numpy as np
# also show the price of previous 30 days + datetime
idx = np.random.randint(0,len(yhat-3))
if idx <30:
  date_actual = date_test[:idx+3]
else:
  date_actual = date_test[idx-30:idx+3]

_=pyplot.plot(date_actual,inv_y[max(0,idx-30):idx+3,0],'b')
_=pyplot.plot(date_test[idx:idx+3],inv_yhat[idx],'r')